In [20]:
include("/Users/kate/github/Kraft.jl/src/Kraft.jl")

Main.Kraft

In [21]:
project_dir = dirname(@__DIR__)

"/Users/kate/github/omics_sample"

In [22]:
input_dir = joinpath(project_dir, "input")

"/Users/kate/github/omics_sample/input"

In [23]:
output_dir = joinpath(project_dir, "output")

"/Users/kate/github/omics_sample/output"

In [24]:
Kraft.check_program()

Checking program...
`which skewer`
/Users/kate/miniconda3/bin/skewer
`which fastqc`
/Users/kate/miniconda3/bin/fastqc
`which bgzip`
/Users/kate/miniconda3/bin/bgzip
`which tabix`
/Users/kate/miniconda3/bin/tabix
`which minimap2`
/Users/kate/miniconda3/bin/minimap2
`which samtools`
/Users/kate/miniconda3/bin/samtools
`which bcftools`
/Users/kate/miniconda3/bin/bcftools
`which kallisto`
/Users/kate/miniconda3/bin/kallisto
`bash -c 'source activate py2 && which configManta.py'`
/Users/kate/miniconda3/envs/py2/bin/configManta.py
`bash -c 'source activate py2 && which configureStrelkaGermlineWorkflow.py'`


LoadError: failed process: Process(`bash -c 'source activate py2 && which configureStrelkaGermlineWorkflow.py'`, ProcessExited(1)) [1]


In [27]:
data_for_processing_sequence_dir = joinpath(input_dir, "sample")

"/Users/kate/github/omics_sample/input/sample"

In [28]:
if !isdir(data_for_processing_sequence_dir)

    Kraft.print_and_run_cmd(`unzip -o -d $input_dir $data_for_processing_sequence_dir.zip`)
    
end

In [31]:
snpeff = joinpath(expanduser("~"), "Downloads", "snpEff", "snpEff.jar")

"/Users/kate/Downloads/snpEff/snpEff.jar"

In [32]:
if !isfile(snpeff)
    
    throw("$snpeff is missing.")
    
end

In [33]:
using JSON: parse

In [34]:
project_json = parse(read(joinpath(project_dir, "project.json"), String))

Dict{String,Any} with 7 entries:
  "soma_dna.1.fastq.gz" => "input/sample/soma_dna.1.1k.fastq.gz"
  "gb_memory"           => 48
  "n_job"               => 8
  "dna_is_targeted"     => false
  "germ_dna.2.fastq.gz" => "input/sample/germ_dna.2.1k.fastq.gz"
  "soma_dna.2.fastq.gz" => "input/sample/soma_dna.2.1k.fastq.gz"
  "germ_dna.1.fastq.gz" => "input/sample/germ_dna.1.1k.fastq.gz"

In [35]:
process_dna_arguments = (
    joinpath(
        data_for_processing_sequence_dir,
        "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.gz",
    ),
    joinpath(data_for_processing_sequence_dir, "chromosome.bed.gz"),
    joinpath(data_for_processing_sequence_dir, "chrn_n.tsv"),
    project_json["n_job"],
    project_json["gb_memory"],
    2,
    snpeff,
)

("/Users/kate/github/omics_sample/input/sample/GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.gz", "/Users/kate/github/omics_sample/input/sample/chromosome.bed.gz", "/Users/kate/github/omics_sample/input/sample/chrn_n.tsv", 8, 48, 2, "/Users/kate/Downloads/snpEff/snpEff.jar")

In [ ]:
if all((
    in(key, keys(project_json)) for key in ("germ_dna.1.fastq.gz", "germ_dna.2.fastq.gz")
))

    Kraft.process_germ_dna(
        joinpath(project_dir, project_json["germ_dna.1.fastq.gz"]),
        joinpath(project_dir, project_json["germ_dna.2.fastq.gz"]),
        project_json["dna_is_targeted"],
        joinpath(output_dir, "process_germ_dna"),
        process_dna_arguments...,
    )

end

In [ ]:
if all((
    in(key, keys(project_json))
    for
    key in (
        "germ_dna.1.fastq.gz",
        "germ_dna.2.fastq.gz",
        "soma_dna.1.fastq.gz",
        "soma_dna.2.fastq.gz",
    )
))

    Kraft.process_soma_dna(
        joinpath(project_dir, project_json["germ_dna.1.fastq.gz"]),
        joinpath(project_dir, project_json["germ_dna.2.fastq.gz"]),
        joinpath(project_dir, project_json["soma_dna.1.fastq.gz"]),
        joinpath(project_dir, project_json["soma_dna.2.fastq.gz"]),
        project_json["dna_is_targeted"],
        joinpath(output_dir, "process_soma_dna"),
        process_dna_arguments...,
    )

end

In [ ]:
if all((
    in(key, keys(project_json)) for key in ("soma_rna.1.fastq.gz", "soma_rna.2.fastq.gz")
))

    Kraft.process_soma_rna(
        joinpath(project_dir, project_json["soma_rna.1.fastq.gz"]),
        joinpath(project_dir, project_json["soma_rna.2.fastq.gz"]),
        joinpath(output_dir, "process_soma_rna"),
        joinpath(data_for_processing_sequence_dir, "Homo_sapiens.GRCh38.cdna.all.fa.gz"),
        project_json["n_job"],
    )

end